# Imports

In [2]:
#export
import sys
sys.path.append('/home/sokolov/work/cycler/crsch_cycle/exp/')

import os
import numpy as np
from pathlib import Path

from functools import partial
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago

from nb_runner import cycle_exp, cycle_mutate, cycle_crossover, cycle_combine, cycle_all

# Code

In [2]:
#export
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': days_ago(2),
    'email': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 0,
    'retry_delay': timedelta(minutes=5),}

default_pool = 'sokolov_pool_gpu0'

dag = DAG(  'crsch_hyp_search1',
                default_args=default_args,
                description='crossover, mutate, combine chances',
                schedule_interval=None)

def base_task_generator(name, func, dag, pool=default_pool, op_kwargs=None):
    task = PythonOperator(
                    task_id=name,
                    python_callable=func,
                    op_kwargs=op_kwargs,
                    provide_context=True,
                    pool=pool,
                    dag=dag,)
    return task

create_task = partial(base_task_generator, dag=dag)

def pull_results(**context):
    upstream_task_results = None
    if context is not None:
        task = context['task']
        upstream_task_ids = task.upstream_task_ids
        upstream_task_results = [context['ti'].xcom_pull(task_ids=task_id) for task_id in upstream_task_ids]
    
    return upstream_task_results

def parse_results(upstream_task_results, first=False):
    aux_cfg_files, docker_results = None, None
    
    if upstream_task_results:
        aux_cfg_files, docker_results = [],[]
        for results in upstream_task_results:
            try:
                aux_cfg_files.append(results['configs'])
            except Exception as e:
                print(e)
            try:
                docker_results.append(results['docker_results'])
            except Exception as e:
                print(e)
    
    if first and aux_cfg_files:
        aux_cfg_files = aux_cfg_files[0]
    
    if first and docker_results:
        docker_results = docker_results[0]
    
    return aux_cfg_files, docker_results

def parse_pool(pool_str):
    return ','.join([i for i in pool_str if i.isdigit()])

In [3]:
#export
def dw_cycle_param(func, first=False, **context):
    aux_cfg_files, _ = parse_results(pull_results(**context), first=first)
    gpus = parse_pool(context['ti'].pool)
    return func(aux_cfg_files=aux_cfg_files, gpus=gpus)

def dw_cycle_pool(**context):
    aux_cfg_files, docker_results = parse_results(pull_results(**context))
    idx = np.argmax(docker_results)
    print(f'\n\tPooling: best result in #{idx}: {docker_results[idx]}, {aux_cfg_files[idx]}\n')
    return {'configs':aux_cfg_files[idx]}

def cycle_block(n, name, func):
    tasks = []
    for i in range(n):
        task_name = f'{name}_{i}'
        pool = f'sokolov_pool_gpu{i%3}'
        func_partial = partial(func, seq_id=i, name=task_name)
        task = create_task(task_name, dw_cycle_param, pool=pool, op_kwargs={'func':func_partial, 'first':True}) 
        tasks.append(task) 
    return tasks

In [5]:
#export
tasks = {'mut':[], 'exp':[], 'cross':[]}

cycle_block(50, 'cycle_all', cycle_all)
#tasks['exp'] = cycle_block(3, 'cycle_e', cycle_exp)

# pooling_task1 = create_task(f'pooling1', dw_cycle_pool) 
# for task in tasks['exp']:
#     task >> pooling_task1

# tasks['mut'] = cycle_block(10, 'cycle_m', cycle_mutate)
# pooling_task2 = create_task(f'pooling2', dw_cycle_pool) 
# for task in tasks['mut']:
#     pooling_task1 >> task
#     task >> pooling_task2
    
# tasks['cross'] = cycle_block(10, 'cycle_cr', cycle_crossover)
# pooling_task3 = create_task(f'pooling3', dw_cycle_pool) 
# for task in tasks['cross']:
#     pooling_task2 >> task
#     task >> pooling_task3
    
# tasks['comb'] = cycle_block(10, 'cycle_co', cycle_combine)
# for task in tasks['comb']:
#     pooling_task3 >> task

In [32]:
from airflow.models import TaskInstance
from datetime import datetime
import json
from pprint import pprint

In [33]:
def test(**kwargs):
    pprint(kwargs)
    return 5

In [34]:
task = base_task_generator('test', test, dag)

In [35]:
ti = TaskInstance(task=task, execution_date=datetime.now())
task.execute(context=ti.get_template_context())

{'END_DATE': '2020-05-10',
 'conf': <airflow.configuration.AirflowConfigParser object at 0x7f204759c1d0>,
 'dag': <DAG: crsch_hyp_search1>,
 'dag_run': None,
 'ds': '2020-05-10',
 'ds_nodash': '20200510',
 'end_date': '2020-05-10',
 'execution_date': <Pendulum [2020-05-10T23:26:41.864886+00:00]>,
 'inlets': [],
 'latest_date': '2020-05-10',
 'macros': <module 'airflow.macros' from '/home/sokolov/.local/lib/python3.6/site-packages/airflow/macros/__init__.py'>,
 'next_ds': None,
 'next_ds_nodash': None,
 'next_execution_date': None,
 'outlets': [],
 'params': {},
 'prev_ds': None,
 'prev_ds_nodash': None,
 'prev_execution_date': None,
 'prev_execution_date_success': <Proxy at 0x7f206fc74e88 with factory <function TaskInstance.get_template_context.<locals>.<lambda> at 0x7f203c545f28>>,
 'prev_start_date_success': <Proxy at 0x7f203c8f87c8 with factory <function TaskInstance.get_template_context.<locals>.<lambda> at 0x7f206ffd7268>>,
 'run_id': None,
 'tables': None,
 'task': <Task(PythonOp

5

In [37]:
ti.pool

'sokolov_pool_6'

In [ ]:
def task_state(args):
    dag = get_dag(args)
    task = dag.get_task(task_id=args.task_id)
    ti = TaskInstance(task, args.execution_date)
    print(ti.current_state())

# Export

In [8]:
!python3 n2s.py dag.ipynb

Converted dag.ipynb to exp/nb_dag.py


In [9]:
!airflow list_dags

[2020-05-10 23:59:44,405] {__init__.py:51} INFO - Using executor LocalExecutor
[2020-05-10 23:59:44,405] {dagbag.py:396} INFO - Filling up the DagBag from /home/sokolov/airflow/dags
[2020-05-10 23:59:44,406] {dagbag.py:396} INFO - Filling up the DagBag from /home/sokolov/work/cycler/crsch_cycle/exp


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
crsch_hyp_search1

